In [24]:
! pip install PyMuPDF requests python-dotenv boto3 opensearch-py requests_aws4auth

  Using cached opensearch_py-2.6.0-py2.py3-none-any.whl.metadata (7.0 kB)
  Using cached Events-0.5-py3-none-any.whl.metadata (3.9 kB)
Using cached opensearch_py-2.6.0-py2.py3-none-any.whl (311 kB)
Using cached Events-0.5-py3-none-any.whl (6.8 kB)

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


### Bedrock & OCR 등 필요 Library 사전 정의

In [3]:
## Upstage OCR 정의
import requests
 
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()
 
## 개인 Key 입니다. Freetier 완료 후 사용 불가능합니다.
## upstage_api.key 파일안에 key를 셋팅해주세요.
api_key = os.getenv("UPSTAGE_API_KEY")

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

def get_ocr(input_image_base64):
    files = {"document": input_image_base64}
    response = requests.post(url, headers=headers, files=files)
    
    # print("--------------------------")
    # print(response)
    # print("--------------------------")
    
    return response.json()["text"]



In [4]:
## Bedrock 정의

import boto3
import json
import base64

#파일 바이트에서 base64로 인코딩된 문자열 가져오기
def get_base64_from_bytes(bytesio):
    img_str = base64.b64encode(bytesio.getvalue()).decode("utf-8")
    return img_str

#InvokeModel API 호출에 대한 문자열화된 요청 본문 가져오기
def get_image_understanding_request_body(prompt, bytesio=None, system_prompt=None):
    input_image_base64 = get_base64_from_bytes(bytesio)
    # print("input_image_base64 = > ",input_image_base64)
    body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg", # this doesn't seem to matter?
                            "data": input_image_base64,
                        },
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ],
            }
        ],
    }
    
    return json.dumps(body)

#Anthropic Claude를 사용하여 응답 생성하기
def get_response_from_model(prompt_content, bytesio, model_id, system_prompt=None):
    session = boto3.Session()
    
    bedrock = session.client(service_name='bedrock-runtime') #Bedrock 클라이언트를 생성합니다
    
    body = get_image_understanding_request_body(prompt_content, bytesio, system_prompt=system_prompt)
        
    response = bedrock.invoke_model(body=body, modelId=model_id, contentType="application/json", accept="application/json")
    
    response_body = json.loads(response.get('body').read()) #응답을 읽습니다
    
    output = response_body['content'][0]['text']
    
    return output


bedrock = boto3.client("bedrock-runtime")
bedrock_model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
embedding_model_id = "amazon.titan-embed-text-v2:0"

def get_llm_output(prompt):
    
    body = json.dumps({
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 1024,
                "temperature" : 0.1,
                "top_p": 0.5,
                "messages": [
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                        ],
                    }
                ],
            }) 

    response = bedrock.invoke_model(
        body=body, 
        modelId=bedrock_model_id,
        accept='application/json',
        contentType='application/json')

    response_body = json.loads(response.get("body").read())
    llm_output = response_body.get("content")[0].get("text")
    return llm_output

def get_embedding_output(query, embedding_dimension = 1024):
    
    try:
        body = {
            "inputText": query,
            "dimensions": embedding_dimension,
            "normalize": True
        }

        response = bedrock.invoke_model(
            body=json.dumps(body), 
            modelId=embedding_model_id,
            accept='application/json',
            contentType='application/json')

        response_body = json.loads(response.get("body").read())
        embedding = response_body.get("embedding")
        return embedding
    except Exception as e:
        print(f"Error: {e}")
        return False


In [5]:
## aoss 정의

from opensearchpy import OpenSearch
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

def getOpenSearchClient(endpoint):
    
    session = boto3.Session()
    region = session.region_name
    service = 'aoss'
    credentials = session.get_credentials()
    awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)
    
    aoss_client = OpenSearch(
        hosts=[{'host': endpoint, 'port': 443}],
        http_auth=awsauth,
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection,
        timeout=6000
    )
    return aoss_client

def deleteIndex(client, index_name):
    # client = getOpenSearchClient()
    response = client.indices.delete(index_name)
    print('\nDeleting index:')
    print(response)
    
    return response

# index 생성 함수
def createIndex(client, index_name, index_schema=None):    
    # client = getOpenSearchClient()

    if index_schema:
        response = client.indices.create(index_name, body=index_schema)
    else:
        response = client.indices.create(index_name)
    print('\nCreating index:')
    print(response)
    
    return response


In [8]:
## 초기 index 생성
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()
aoss_endpoint = os.getenv("AOSS_ENDPOINT")
# aoss_collection=os.getenv("AOSS_COLLECTION")
aoss_vector_index=os.getenv("AOSS_VECTOR_INDEX")

# aoss client 생성
aoss_client = getOpenSearchClient(aoss_endpoint)

# 인덱스 생성

# index 생성 스키마 정보
ef_search = 512
embedding_model_dimensions = 1024

index_schema = {
        "settings": {
            "index": {
                "knn": True,
                "knn.algo_param.ef_search": ef_search,
            }
        },
        "mappings": {
            "properties": {
                "content_embeddings": {
                    "type": "knn_vector",
                    "dimension": embedding_model_dimensions,
                    "method": {
                        "name": "hnsw",
                        "space_type": "cosinesimil",
                        # "space_type": "l2",
                        "engine": "nmslib",
                        "parameters": {"ef_construction": 512, "m": 16},
                    },
                },
                "content": {"type": "text", "analyzer": "nori"},
                "metadata": {"type": "object"},
            }
        },
    } 

deleteIndex(aoss_client, aoss_vector_index)
createIndex(aoss_client, aoss_vector_index, index_schema)



Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag-nice-index-vector'}


{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'rag-nice-index-vector'}

In [9]:
## index 조회
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()
aoss_endpoint = os.getenv("AOSS_ENDPOINT")
# aoss_collection=os.getenv("AOSS_COLLECTION")
aoss_vector_index=os.getenv("AOSS_VECTOR_INDEX")

# aoss client 생성
aoss_client = getOpenSearchClient(aoss_endpoint)

# 인덱스 조회
response = aoss_client.indices.get(aoss_vector_index)
print(response)


{'rag-nice-index-vector': {'aliases': {}, 'mappings': {'properties': {'content': {'type': 'text', 'analyzer': 'nori'}, 'content_embeddings': {'type': 'knn_vector', 'dimension': 1024, 'method': {'engine': 'nmslib', 'space_type': 'cosinesimil', 'name': 'hnsw', 'parameters': {'ef_construction': 512, 'm': 16}}}, 'metadata': {'type': 'object'}}}, 'settings': {'index': {'number_of_shards': '2', 'knn.algo_param': {'ef_search': '512'}, 'provided_name': 'rag-nice-index-vector', 'knn': 'true', 'creation_date': '1722037719820', 'number_of_replicas': '0', 'uuid': 'ephy8ZABXD0ki0lIUvPP', 'version': {'created': '135217827'}}}}}


### PDF Image로 변환

In [10]:
import fitz  # PyMuPDF
import os
from io import BytesIO

def get_image_description(ocr_text,bytes_io):
    prompt_content = """
    <지시사항>
    - 당신은 상권정보 리포트를 아주 자세히 서술해주는 로봇입니다.
    - 첨부된 이미지는 상권정보를 분석하는 PDF의 하나의 페이지입니다.
    - 첨부된 이미지와 <ocr> 정보를 바탕으로 해당 이미지를 아주 자세히 설명하는 Markdown 형태의 장문의 문서를 만들어주세요.
    - <ocr>에 있는 모든 Text는 응답결과에 반드시 모두 포함되어야 합니다.
    - 결과는 반드시 한글로 응답해주세요.
    </지시사항>
    
    <필수 데이터 관련 지시사항>
    - 해당 이미지에 테이블 형태에 데이터가 포함되어 있다면 <ocr>을 바탕으로 Markdown 형식으로 테이블 형식으로 데이터를 추출해주세요.
    - 해당 이미지에 차트가 포함되어 있다면 차트에 포함된 추세와 차트내 데이터를 자세히 분석 및 서술해주세요.
    - 해당 지도에 정보가 포함되어 있다면 지도내 읍/면/동 등 지역과 그 지역의 상권정보를 자세히 분석 및 서술해주세요.
    </필수 데이터 관련 지시사항>

    
    <ocr/>
    """

    prompt_content = prompt_content.replace(
        "<ocr/>",
        """<ocr>"""+ocr_text+"""</ocr>"""
    )

    modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
    content = get_response_from_model(prompt_content=prompt_content, bytesio=bytes_io, model_id=modelId)

    return content

def pdf_to_vectorstore(pdf_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        # if(page_num == 32):
        
        # text = page.get_text()
        
        ## 1. 각 페이지를 image 추출
        pix = page.get_pixmap()
        
        ## 2. ocr text 추출
        img_bytes = pix.tobytes()
        bytes_io = BytesIO(img_bytes)
        byte_string = bytes_io.getvalue()
        ocr_text = get_ocr(byte_string)

        ## 3. get image description
        content = get_image_description(ocr_text, bytes_io)
        
        ## 4. aoss indexting
        meta = {"page_num" : page_num}
        embedding = get_embedding_output(content)

        data = {
            "content": content,
            "content_embeddings": embedding,
            "metadata": meta,
        }

        response = aoss_client.index(index=aoss_vector_index, body=data)        
        # 페이지 번호와 함께 텍스트 출력
        print(f"--- Page {page_num + 1} ---")
        print(response)
        # print("\n")  # 페이지 간 구분을 위한 빈 줄

    doc.close()

# 사용 예시
pdf_path = "/Users/dhoon/Desktop/나이스_RAG/나이스비즈맵_상세보고서.pdf"
output_folder = "./pdf_images"

pdf_to_vectorstore(pdf_path, output_folder)

--- Page 1 ---
{'_index': 'rag-nice-index-vector', '_id': '1%3A0%3AGvhy8ZABS2cHzgBr8XIN', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
--- Page 2 ---
{'_index': 'rag-nice-index-vector', '_id': '1%3A0%3A93Vz8ZAB_qttyzNoNN3d', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
--- Page 3 ---
{'_index': 'rag-nice-index-vector', '_id': '1%3A0%3AG_hz8ZABS2cHzgBri3Iv', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
--- Page 4 ---
{'_index': 'rag-nice-index-vector', '_id': '1%3A0%3A-HVz8ZAB_qttyzNo_d1S', '_version': 1, 'result': 'created', '_shards': {'total': 0, 'successful': 0, 'failed': 0}, '_seq_no': 0, '_primary_term': 0}
--- Page 5 ---
{'_index': 'rag-nice-index-vector', '_id': '1%3A0%3AHPh08ZABS2cHzgBrVnKT', '_version': 1, 'result': 'created', '_shards': {'t